In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd


In [2]:
def open_selection():
    driver.find_element(By.XPATH, '//*[@class="gwt-Button GNKVYU1OU"]').click()
    time.sleep(1)
    seznam = driver.find_elements(By.XPATH, '//*[@class="gwt-InlineLabel"]')
    for item in seznam:
        if 'Programme of study' in item.text:
            item.click()
            break

In [3]:
# Function to open dropdown menu of faculties
def open_dropdown():
    buttons = driver.find_elements(By.XPATH, '//*[@class="gwt-Label GNKVYU1OR"]')
    buttons[1].click()


In [ ]:
# Function to get courses from individual faculties
def get_courses():
    faculties = driver.find_elements(By.XPATH, '//div[@class="GNKVYU1P3"]/ul/li')
    for faculty in faculties:
        faculty.click()
        time.sleep(1)
        courses = driver.find_elements(By.XPATH, '//div[@class="GNKVYU1AQ"]')
        for course in courses:
            courses_list.append(course.text)
        time.sleep(1)
        open_dropdown()
    return courses_list


In [74]:
def add_timetables():
    i = 0

    faculties = driver.find_elements(By.XPATH, '//div[@class="GNKVYU1P3"]/ul/li')
    for faculty in faculties:
        end = False   
        # Click next option in dropdown menu
        faculty.click()
        time.sleep(1)
        courses = driver.find_elements(By.XPATH, '//div[@class="GNKVYU1AQ"]/span')
        for course in courses:
            if course.text in courses_25 and i < 25:   
                # Select course
                course.click()
                i+=1
            if i >= 25:  # Stop selecting once 25 courses have been selected
                break       
        # Add timetables  
        driver.find_element(By.XPATH, '//button[@class="gwt-Button GNKVYU1NK"]').click()
        if i >= 25:
            break                        
        time.sleep(1)
        open_dropdown()
    time.sleep(1)
    # Close window
    driver.find_element(By.XPATH, '//button[@class="gwt-Button GNKVYU1PK"]').click()
    time.sleep(1)


In [72]:
# Scrape events from calendar view
def get_calendar():
    actions = ActionChains(driver)

    nrfailed = 0
    events = driver.find_elements(By.XPATH, '//div[@class="wc-cal-event"]')
    for event in events:
        event.click()
        time.sleep(1)
        try:
            date = driver.find_element(By.XPATH, '//div[@class="gwt-Label GNKVYU1OS"]').text
            lesson_time = driver.find_element(By.XPATH, '//div[@class="gwt-Label GNKVYU1PS"]').text
            fields = driver.find_elements(By.XPATH, '//div[@class="GNKVYU1DS"]/div/div/div')
            for field in fields:
                if 'Course code' in field.text:
                    code = field.text
                if 'Location(s)' in field.text:
                    location = field.text
                if 'Size' in field.text:
                    size = field.text
            time.sleep(1)
            event_dict = {'date': date, 'lesson_time': lesson_time, 'code': code, 'location': location, 'size': size}
            calendar_events.append(event_dict)
        except Exception as e:
                nrfailed+=1
        actions.send_keys(Keys.ESCAPE).perform()
    return calendar_events, nrfailed

In [ ]:
# Get first 3 events from calendar view
def get_calendar_test():
    actions = ActionChains(driver)

    nrfailed = 0
    counter = 0
    events = driver.find_elements(By.XPATH, '//div[@class="wc-time"]')
    for event in events:
        counter += 1
        event.click()
        time.sleep(1)
        try:
            date = driver.find_element(By.XPATH, '//div[@class="gwt-Label GNKVYU1OS"]').text
            lesson_time = driver.find_element(By.XPATH, '//div[@class="gwt-Label GNKVYU1PS"]').text
            fields = driver.find_elements(By.XPATH, '//div[@class="GNKVYU1DS"]/div/div/div')
            for field in fields:
                if 'Course code' in field.text:
                    code = field.text
                if 'Location(s)' in field.text:
                    location = field.text
                if 'Size' in field.text:
                    size = field.text
            time.sleep(1)
            event_dict = {'date': date, 'lesson_time': lesson_time, 'code': code, 'location': location, 'size': size}
            calendar_events.append(event_dict)
        except Exception as e:
                nrfailed+=1
        actions.send_keys(Keys.ESCAPE).perform()
        if counter == 3:
            break
    return calendar_events, nrfailed

In [8]:
def remove_calendars():
    driver.find_element(By.XPATH, '//div[@class="GNKVYU1FJ"][1]').click()
    time.sleep(1)
    closers = driver.find_elements(By.XPATH, '//li[@class="GNKVYU1N2"]')
    for item in closers:
        if 'Remove all' in item.text:
            item.click()
            break
    buttons = driver.find_elements(By.XPATH, '//div[@class="GNKVYU1HO"]/button')
    for button in buttons:
        if 'Remove all' in button.text:
            button.click()
            break

In [ ]:
# Open rooster

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://rooster.uva.nl/schedule")
time.sleep(3)

In [ ]:
# Get list of courses

open_selection()
open_dropdown()
courses_list = []
get_courses()

['Enter a search term to start searching.',
 'Bachelor Liberal Arts and Sciences (joint degree)\nBachelor Liberal Arts and Sciences (joint degree), Academic Core\nBachelor Liberal Arts and Sciences (joint degree), major Humanities\nBachelor Liberal Arts and Sciences (joint degree), major Sciences\nBachelor Liberal Arts and Sciences (joint degree), major Social Sciences\nExch B AUC',
 "Bachelor's in Actuarial Science year 1\nBachelor's in Actuarial Science year 2\nBachelor's in Actuarial Science year 3\nBachelor's in Business Administration year 1, Shared programme\nBachelor's in Business Administration year 2, Shared programme\nBachelor's in Business Administration year 3, Specialisation Accounting & Control\nBachelor's in Business Administration year 3, Specialisation Entrepreneurship, Innovation & Creativity\nBachelor's in Business Administration year 3, Specialisation Finance\nBachelor's in Business Administration year 3, Specialisation Management in the Digital Age\nBachelor's in B

In [ ]:
# Delte first element of list, since it's not a course
new_list = courses_list[1:]

In [62]:
print(new_list)

['Bachelor Liberal Arts and Sciences (joint degree)\nBachelor Liberal Arts and Sciences (joint degree), Academic Core\nBachelor Liberal Arts and Sciences (joint degree), major Humanities\nBachelor Liberal Arts and Sciences (joint degree), major Sciences\nBachelor Liberal Arts and Sciences (joint degree), major Social Sciences\nExch B AUC', "Bachelor's in Actuarial Science year 1\nBachelor's in Actuarial Science year 2\nBachelor's in Actuarial Science year 3\nBachelor's in Business Administration year 1, Shared programme\nBachelor's in Business Administration year 2, Shared programme\nBachelor's in Business Administration year 3, Specialisation Accounting & Control\nBachelor's in Business Administration year 3, Specialisation Entrepreneurship, Innovation & Creativity\nBachelor's in Business Administration year 3, Specialisation Finance\nBachelor's in Business Administration year 3, Specialisation Management in the Digital Age\nBachelor's in Business Analytics year 1\nBachelor's in Busin

In [ ]:
# Flatten list, select frist 25 courses
new_list = [course.split('\n') for course in new_list]
flat_list = [item for sublist in new_list for item in sublist]
courses_25 = flat_list[:25]
print(courses_25)

['Bachelor Liberal Arts and Sciences (joint degree)', 'Bachelor Liberal Arts and Sciences (joint degree), Academic Core', 'Bachelor Liberal Arts and Sciences (joint degree), major Humanities', 'Bachelor Liberal Arts and Sciences (joint degree), major Sciences', 'Bachelor Liberal Arts and Sciences (joint degree), major Social Sciences', 'Exch B AUC', "Bachelor's in Actuarial Science year 1", "Bachelor's in Actuarial Science year 2", "Bachelor's in Actuarial Science year 3", "Bachelor's in Business Administration year 1, Shared programme", "Bachelor's in Business Administration year 2, Shared programme", "Bachelor's in Business Administration year 3, Specialisation Accounting & Control", "Bachelor's in Business Administration year 3, Specialisation Entrepreneurship, Innovation & Creativity", "Bachelor's in Business Administration year 3, Specialisation Finance", "Bachelor's in Business Administration year 3, Specialisation Management in the Digital Age", "Bachelor's in Business Analytics

In [69]:
print(courses_25)

["Research master's in Urban Studies year 1", "Research master's in Urban Studies year 2"]


In [68]:
print(len(flat_list))

2


In [66]:
calendar_events = []

In [ ]:
# Run everything

while len(flat_list) > 25:
    try:
        add_timetables()
        time.sleep(3)
        get_calendar_test()
        remove_calendars()
        # Delete first 25 courses from list, select next 25
        del flat_list[:25]
        courses_25 = flat_list[:25]
        open_selection()
        open_dropdown()
    except Exception as e:
        driver.refresh()
        time.sleep(3)
        open_selection()
        open_dropdown()
        continue

In [ ]:
# Turn data into dataframe and export to csv

df = pd.DataFrame(calendar_events)

df['location'] = df['location'].str.split('\n').str[1]
df['size'] = df['size'].str.split('\n').str[1]
df['code'] = df['code'].str.split('\n').str[1]
df

df.to_csv('calendar_events.csv', index=False)